In [21]:
import subprocess
import sys

for package in ["keras", "numpy", "pandas", "tensorflow", "scikit-learn"]:
	subprocess.check_call([sys.executable, "-m", "pip", "install", package])

import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.regularizers import l2
from keras.optimizers import Adam

# Leer el dataset
df = pd.read_csv('movie_statistic_dataset.csv')

# Analizar los datos
df.head()


,movie_title,production_date,genres,runtime_minutes,director_name,director_professions,director_birthYear,director_deathYear,movie_averageRating,movie_numerOfVotes,approval_Index,Production budget $,Domestic gross $,Worldwide gross $
0,Avatar: The Way of Water,2022-12-09,"Action,Adventure,Fantasy",192.0,James Cameron,"writer,producer,director",1954,alive,7.8,277543.0,7.061101,460000000,667830256,2265935552
1,Avengers: Endgame,2019-04-23,"Action,Adventure,Drama",181.0,-,-,-,-,8.4,1143642.0,8.489533,400000000,858373000,2794731755
2,Pirates of the Caribbean: On Stranger Tides,2011-05-20,"Action,Adventure,Fantasy",137.0,Rob Marshall,"director,miscellaneous,producer",1960,alive,6.6,533763.0,6.272064,379000000,241071802,1045713802
3,Avengers: Age of Ultron,2015-04-22,"Action,Adventure,Sci-Fi",141.0,Joss Whedon,"writer,producer,director",1964,alive,7.3,870573.0,7.214013,365000000,459005868,1395316979
4,Avengers: Infinity War,2018-04-25,"Action,Adventure,Sci-Fi",149.0,-,-,-,-,8.4,1091968.0,8.460958,300000000,678815482,2048359754


In [22]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4380 entries, 0 to 4379
Data columns (total 14 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   movie_title           4380 non-null   object 
 1   production_date       4380 non-null   object 
 2   genres                4380 non-null   object 
 3   runtime_minutes       4380 non-null   float64
 4   director_name         4380 non-null   object 
 5   director_professions  4380 non-null   object 
 6   director_birthYear    4380 non-null   object 
 7   director_deathYear    4380 non-null   object 
 8   movie_averageRating   4380 non-null   float64
 9   movie_numerOfVotes    4380 non-null   float64
 10  approval_Index        4380 non-null   float64
 11  Production budget $   4380 non-null   int64  
 12  Domestic gross $      4380 non-null   int64  
 13  Worldwide gross $     4380 non-null   int64  
dtypes: float64(4), int64(3), object(7)
memory usage: 479.2+ KB


In [23]:
df = df.dropna()

# Seleccionar características y objetivo
features = ['runtime_minutes', 'movie_averageRating', 'movie_numerOfVotes', 'approval_Index', 'Production budget $', 'Domestic gross $', 'genres']
target = 'Worldwide gross $'

X = df[features]
y = df[target]

# Preprocesamiento de datos
numeric_features = ['runtime_minutes', 'movie_averageRating', 'movie_numerOfVotes', 'approval_Index', 'Production budget $', 'Domestic gross $']
categorical_features = ['genres']

numeric_transformer = StandardScaler()
categorical_transformer = OneHotEncoder(handle_unknown='ignore')

preprocessor = ColumnTransformer(
	transformers=[
		('num', numeric_transformer, numeric_features),
		('cat', categorical_transformer, categorical_features)
	])

# Crear y dividir los datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Aplicar la transformación
X_train = preprocessor.fit_transform(X_train)
X_test = preprocessor.transform(X_test)

print("X_train shape:", X_train.shape)
print("y_train shape:", y_train.shape)
print("X_test shape:", X_test.shape)
print("y_test shape:", y_test.shape)


X_train shape: (3504, 318)
y_train shape: (3504,)
X_test shape: (876, 318)
y_test shape: (876,)


In [24]:
# Modelo 1
model1 = Sequential([
	Dense(64, input_dim=X_train.shape[1], activation='relu'),
	Dropout(0.2),
	Dense(32, activation='relu'),
	Dense(1, activation='linear')
])
model1.compile(optimizer=Adam(learning_rate=0.001), loss='mse', metrics=['mae'])
# Entrenar
history1 = model1.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=50, batch_size=32)
# Evaluar
loss1, mae1 = model1.evaluate(X_test, y_test)
print(f'Model 1 - Loss: {loss1}, MAE: {mae1}')

c:\Programs\Coding\Python\Lib\site-packages\keras\src\layers\core\dense.py:85: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


ValueError: Unrecognized data type: x=  (0, 0)	0.29970697015284004
  (0, 1)	-0.09678766554139802
  (0, 2)	-0.5928605320490684
  (0, 3)	-0.861358431775164
  (0, 4)	-0.42188832552082833
  (0, 5)	-0.676317798355686
  (0, 213)	1.0
  (1, 0)	0.20264844673180937
  (1, 1)	-0.19427084463224586
  (1, 2)	-0.5142687038047253
  (1, 3)	-0.4196995368610382
  (1, 4)	-0.18046476470984202
  (1, 5)	-0.01733463218874696
  (1, 274)	1.0
  (2, 0)	0.6879410638369625
  (2, 1)	0.6830777671853873
  (2, 2)	0.04985461290148334
  (2, 3)	0.7676401925017781
  (2, 4)	0.2584871640374058
  (2, 5)	0.32809423234075225
  (2, 161)	1.0
  (3, 0)	0.7364703255474779
  (3, 1)	0.5855945880945395
  (3, 2)	-0.10955623127152007
  (3, 3)	0.6026844472007026
  :	:
  (3500, 3)	-0.3802021022770585
  (3500, 4)	1.1363910215319015
  (3500, 5)	0.12862112949279475
  (3500, 121)	1.0
  (3501, 0)	0.5908825404159319
  (3501, 1)	1.0730104835487804
  (3501, 2)	-0.1731888584575421
  (3501, 3)	0.8577895904471136
  (3501, 4)	-0.5974690970197275
  (350

In [ ]:
# Modelo 2
model2 = Sequential([
	Dense(128, input_dim=X_train.shape[1], activation='tanh', kernel_regularizer=l2(0.01)),
	Dense(64, activation='tanh', kernel_regularizer=l2(0.01)),
	Dense(1, activation='linear')
])
model2.compile(optimizer=Adam(learning_rate=0.001), loss='mse', metrics=['mae'])
# Entrenar
history2 = model2.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=50, batch_size=32)
# Evaluar
loss2, mae2 = model2.evaluate(X_test, y_test)
print(f'Model 2 - Loss: {loss2}, MAE: {mae2}')

ValueError: Unrecognized data type: x=  (0, 0)	0.29970697015284004
  (0, 1)	-0.09678766554139802
  (0, 2)	-0.5928605320490684
  (0, 3)	-0.861358431775164
  (0, 4)	-0.42188832552082833
  (0, 5)	-0.676317798355686
  (0, 213)	1.0
  (1, 0)	0.20264844673180937
  (1, 1)	-0.19427084463224586
  (1, 2)	-0.5142687038047253
  (1, 3)	-0.4196995368610382
  (1, 4)	-0.18046476470984202
  (1, 5)	-0.01733463218874696
  (1, 274)	1.0
  (2, 0)	0.6879410638369625
  (2, 1)	0.6830777671853873
  (2, 2)	0.04985461290148334
  (2, 3)	0.7676401925017781
  (2, 4)	0.2584871640374058
  (2, 5)	0.32809423234075225
  (2, 161)	1.0
  (3, 0)	0.7364703255474779
  (3, 1)	0.5855945880945395
  (3, 2)	-0.10955623127152007
  (3, 3)	0.6026844472007026
  :	:
  (3500, 3)	-0.3802021022770585
  (3500, 4)	1.1363910215319015
  (3500, 5)	0.12862112949279475
  (3500, 121)	1.0
  (3501, 0)	0.5908825404159319
  (3501, 1)	1.0730104835487804
  (3501, 2)	-0.1731888584575421
  (3501, 3)	0.8577895904471136
  (3501, 4)	-0.5974690970197275
  (350

In [ ]:
# Modelo 3
model3 = Sequential([
	Dense(256, input_dim=X_train.shape[1], activation='relu', kernel_regularizer=l2(0.01)),
	Dropout(0.2),
	Dense(128, activation='relu', kernel_regularizer=l2(0.01)),
	Dropout(0.2),
	Dense(64, activation='relu', kernel_regularizer=l2(0.01)),
	Dense(1, activation='linear')
])
model3.compile(optimizer=Adam(learning_rate=0.001), loss='mse', metrics=['mae'])
# Entrenar
history3 = model3.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=50, batch_size=32)
# Evaluar
loss3, mae3 = model3.evaluate(X_test, y_test)
print(f'Model 3 - Loss: {loss3}, MAE: {mae3}')

ValueError: Unrecognized data type: x=  (0, 0)	0.29970697015284004
  (0, 1)	-0.09678766554139802
  (0, 2)	-0.5928605320490684
  (0, 3)	-0.861358431775164
  (0, 4)	-0.42188832552082833
  (0, 5)	-0.676317798355686
  (0, 213)	1.0
  (1, 0)	0.20264844673180937
  (1, 1)	-0.19427084463224586
  (1, 2)	-0.5142687038047253
  (1, 3)	-0.4196995368610382
  (1, 4)	-0.18046476470984202
  (1, 5)	-0.01733463218874696
  (1, 274)	1.0
  (2, 0)	0.6879410638369625
  (2, 1)	0.6830777671853873
  (2, 2)	0.04985461290148334
  (2, 3)	0.7676401925017781
  (2, 4)	0.2584871640374058
  (2, 5)	0.32809423234075225
  (2, 161)	1.0
  (3, 0)	0.7364703255474779
  (3, 1)	0.5855945880945395
  (3, 2)	-0.10955623127152007
  (3, 3)	0.6026844472007026
  :	:
  (3500, 3)	-0.3802021022770585
  (3500, 4)	1.1363910215319015
  (3500, 5)	0.12862112949279475
  (3500, 121)	1.0
  (3501, 0)	0.5908825404159319
  (3501, 1)	1.0730104835487804
  (3501, 2)	-0.1731888584575421
  (3501, 3)	0.8577895904471136
  (3501, 4)	-0.5974690970197275
  (350